![alt text][logo]

[logo]:https://wri-public-data.s3.amazonaws.com/resourcewatch/18_LOGO_ResourceWatch/18_LOGO_ResourceWatch_PB.png "Logo Title Text 2"

#Resource Watch Query Tutorial

In [0]:
#@title # **Pemasangan Alat Bantu** { display-mode: "form" }
#@markdown ###Tekan tombol “play” di sebelah kiri untuk memasang alat bantu bagi analisis Anda.
import urllib
import urllib.parse
import requests
import pandas as pd
print('Selesai menginstal modul!')
print('Anda dapat beralih ke sel berikutnya.')

# Ikhtisar
Alat bantu analisis sumber terbuka Resource Watch ini memungkinkan Anda untuk melihat keseluruhan dataset yang disimpan dalam platform, atau bagian tertentu dari dataset, dengan cepat dan mudah. Anda juga dapat mengambil statistik sederhana dari data pada tingkat global, atau dalam wilayah minat Anda.
Alat bantu ini memanfaatkan Antarmuka Pemrograman Aplikasi Resource Watch, atau API, yakni infrastruktur back-end situs resourcewatch.org yang memungkinkan Anda membagikan, memvisualisasikan, dan menganalisis ratusan dataset.
<br><br>
Alat bantu ini merupakan versi tutorial dari alat bantu lengkap, dan menggunakan rangkaian contoh untuk mengajarkan dasar penting dalam penulisan Resource Watch API queries. API queries didasarkan pada sistem manajemen basis data bernama PostgreSQL, yang merupakan versi sumber terbuka _Structured Query Language_ (SQL).
<br><br>
Untuk membuat query dalam API, Anda harus memiliki dua informasi utama: Resource Watch dataset ID dan pernyataan SQL yang menunjukkan informasi yang Anda inginkan dari dataset. Pertama-tama, Anda perlu membuka https://resourcewatch.org, lalu klik tombol 'Explore data' di laman depan. Pada laman 'Explore', Anda dapat mencari berbagai dataset yang dapat digunakan dalam alat bantu analisis data ini.


#Mendapatkan Dataset ID

Dataset ID merupakan informasi pertama yang perlu dianalisis menggunakan alat bantu ini. Dataset ID dapat ditemukan dalam laman metadata dataset. Setiap dataset yang ada di Resource Watch memiliki laman metadata tersendiri. Terdapat dua cara untuk membuka laman metadata dari katalog 'Explore':

1) Anda dapat mengeklik tombol 'Details' pada kartu dataset dalam katalog 'Explore'.

![alt text][sshot1]

[sshot1]:http://wri-public-data.s3.amazonaws.com/resourcewatch/Screenshots/Indonesia%20Workshop/Details.png

2) Jika dataset telah ditambahkan ke peta Anda dalam 'Explore', Anda dapat mengeklik tombol 'i' pada keterangan, lalu klik tombol 'More info' pada jendela pop-up.

![alt text][sshot2]

[sshot2]:http://wri-public-data.s3.amazonaws.com/resourcewatch/Screenshots/Indonesia%20Workshop/Layer%20Info.png

![alt text][sshot3]

[sshot3]:http://wri-public-data.s3.amazonaws.com/resourcewatch/Screenshots/Indonesia%20Workshop/More%20Info.png

Setelah Anda membuka laman metadata dataset, Anda dapat memperoleh Dataset ID dari bagian akhir url: resourcewatch.org/data/explore/{dataset_id}. Nantinya, Anda perlu menyalin dan menempelkan ID ini ke alat bantu, jadi tetap buka tab ini.

![alt text][sshot4]

[sshot4]:http://wri-public-data.s3.amazonaws.com/resourcewatch/Screenshots/Indonesia%20Workshop/Dataset%20ID.png

#Memulai dengan SQL

Semua dataset vektor aktif Resource Watch disimpan dalam tabel berupa baris dan kolom. Nama kolom menunjukkan informasi yang tengah disimpan dalam tabel, dan setiap baris menunjukkan geografi tertentu. Data yang disimpan dalam tabel dapat terdiri atas empat jenis: teks (text, juga disebut string), angka (number), geometri (geometry), atau tanggal (date, juga disebut timestamp). Setiap kolom harus diberi label sebagai salah satu dari empat jenis data tersebut, dan kolom ini hanya dapat menyimpan jenis informasi tersebut. Perhatikan bahwa string/kolom teks dapat menyimpan teks dan angka, namun statistik hanya dapat dihitung untuk kolom yang ditetapkan sebagai kolom angka.

Anda dapat melihat contoh salah satu tabel ini pada tangkapan layar di bawah ini. Perhatikan bahwa dalam nama setiap kolom menunjukkan jenis kolom.

![alt text][sshot4]

[sshot4]:http://wri-public-data.s3.amazonaws.com/resourcewatch/Screenshots/Indonesia%20Workshop/Carto%20table.png

Sebagaimana disebutkan di atas, SQL merupakan sistem manajemen basis data. SQL merupakan bahasa yang digunakan untuk menarik informasi tertentu dari tabel seperti ini. Misalnya, jika saya hanya ingin melihat data dari satu lokasi atau jika saya hanya ingin melihat nilai yang berada di atas ambang batas tertentu, saya dapat menggunakan SQL untuk menarik informasi ini tanpa harus mencari seluruh tabel dan menemukannya secara manual. Ini sangat membantu saat mengerjakan dataset yang memiliki ribuan baris data yang tidak mungkin kita cari sendiri!

Kita akan memulai dengan mempelajari struktur SQL queries dan cara menarik bagian informasi tertentu. Setelah memahami dasar ini, kita akan mengakhirinya dengan mempelajari cara menghitung statistik guna membantu kita menarik informasi bermakna dari dataset besar.

In [0]:
#@markdown ##Queries Dasar
#@markdown 
#@markdown Semua SQL queries memiliki struktur dasar yang wajib Anda mulai:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT {columns} FROM {table_name}
#@markdown 
#@markdown saat {table_name} harus diganti dengan nama tabel yang menyimpan dataset yang diminati. Untuk API Resource Watch, Dataset ID dapat digunakan sebagai {table_name}. Ganti {columns} dengan nama kolom tertentu dalam dataset yang ingin Anda lihat. Terdapat dua opsi untuk memilih kolom yang diminati: Anda dapat memilih semuanya, atau Anda dapat memilih bagian khusus. Untuk memilih semua kolom, Anda dapat mengganti {columns} dengan tanda bintang (*). Untuk memilih bagian data, ganti {columns} dengan nama kolom yang diminati, dipisahkan dengan koma (misalnya: negara, level, nilai).
#@markdown 
#@markdown 
#@markdown ###Contoh:
#@markdown 
#@markdown Mari ambil contoh dataset untuk berlatih. Anda akan bekerja dengan Global Power Plant Database, yang memiliki Dataset ID berikut:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Powerwatch
#@markdown 
#@markdown SQL query dasar Anda akan menjadi:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT * FROM Powerwatch
#@markdown 
#@markdown SQL query akan mengembalikan seluruh tabel data (semua kolom) dalam dataset tersebut dari API, yang akan sangat panjang. Apabila Anda hanya ingin melihat pratinjau data, Anda dapat membatasi jumlah baris data yang ingin dilihat. Anda dapat membatasi hasil hingga 5 baris dengan menambahkan LIMIT 5 pada akhir query. Query ini akan menjadi:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT * FROM Powerwatch LIMIT 5
#@markdown 
#@markdown Anda dapat mengubah angka 5 menjadi nomor lain jika Anda ingin melihat jumlah baris yang berbeda, atau Anda dapat menghapus semuanya sekaligus jika Anda ingin melihat seluruh tabel data.
#@markdown 
#@markdown 

#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.

query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))
  
r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
  
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Queries yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT * FROM Powerwatch
#@markdown <br>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT * FROM Powerwatch LIMIT 5
#@markdown 
#@markdown Anda dapat mengubah query pada bidang di atas dan kembali menjalankan sel untuk mengubah hasil Anda. Coba ubah batas jumlah baris untuk melihat perubahan hasilnya.
#@markdown 
#@markdown **Catatan**: Anda membatasi hasil hanya untuk 5 baris pertama, sehingga Anda tidak mencetak ribuan baris di layar Anda. Apabila Anda mengeklik 'Data Download Link' pada query seperti ini, Anda hanya akan mengunduh 5 baris pertama. Alat bantu ini hanya akan memberi Anda informasi yang diminta. Apabila Anda ingin menganalisis bagian data atau mengunduh hasil lengkap query Anda, pastikan untuk menghapus batas query Anda.

In [0]:
#@markdown ##Memilih kolom tertentu
#@markdown Queries Anda baru memberikan pratinjau dari Basis Data Powerplant Global. Anda mungkin memperhatikan bahwa beberapa kolom berisi informasi yang tidak bermanfaat untuk analisis. Anda dapat mengabaikan kolom ini dan berfokus pada kolom yang berisi informasi bermanfaat dengan mengganti * dengan nama kolom yang ingin Anda lihat. Anda dapat melihat kolom negara, level, wilayah, nilai, dan tahun menggunakan SQL query berikut:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch LIMIT 5


#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.

query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))
  
r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
  
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Queries yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch LIMIT 5

#Filter

In [0]:
#@markdown ##Filter Dasar
#@markdown Setelah Anda mengetahui cara menggunakan alat bantu ini untuk menarik data dari API, sekarang Anda akan mempelajari cara memfilter bagian data sehingga Anda hanya melihat bagian yang berkaitan dengan penelitian Anda. Perintah WHERE dapat ditambahkan ke pernyataan SQL untuk memfilter poin data yang termasuk dalam ketentuan tertentu. Ketentuan ini ditambahkan pada akhir pernyataan SQL Anda:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT {columns} FROM
#@markdown {table_name} WHERE {condition}
#@markdown ###Contoh:

#@markdown Anda mungkin menemukan contoh dataset yang memuat data dari seluruh dunia. Apabila Anda hanya tertarik untuk melihat data dari Indonesia, gunakan kolom 'country_long' untuk hanya menarik data yang berasal dari Indonesia. Tambahkan ketentuan kolom ke pernyataan SQL, WHERE country_long = 'Indonesia' dimana lengkapnya sekarang terlihat seperti ini:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long = 'Indonesia' LIMIT 5
#@markdown 
#@markdown SQL query di atas disusun sebagai berikut:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT {columns} FROM {table_name} WHERE {column_name} = {value}
#@markdown
#@markdown Dalam query di atas, tanda sama dengan (=) dikenal sebagai operator pembanding. Anda tidak selalu harus menggunakan sama dengan (=) sebagai operator Anda. Saat menemukan kolom yang berisi angka, Anda juga dapat menggunakan operator perbandingan, misalnya lebih besar dari (>), lebih besar dari atau sama dengan (>=), kurang dari (<), kurang dari atau sama dengan (<=), atau tidak sama dengan (!=). Apabila Anda ingin menemukan semua data dengan kapasitas lebih besar atau sama dengan 100 megawatt (MW), Anda dapat melakukannya dengan mengikuti query berikut:
#@markdown
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE capacity_mw>=100 LIMIT 5
#@markdown
#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))
  
r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Queries yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long = 'Indonesia' LIMIT 5
#@markdown <br>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE capacity_mw>=100 LIMIT 5
#@markdown <br>

#@markdown **Catatan**: Saat memfilter data dengan SQL query, nama kolom (misalnya country_long) tidak perlu dituliskan dalam tanda kutip, namun nilai (seperti ‘Indonesia’) sering kali perlu ditulis demikian. Saat memfilter kolom berupa teks atau tanggal/catatan waktu, nilai yang Anda cari haruslah berada dalam tanda kutip tunggal (misalnya: ‘New York’). Namun demikian, Anda tidak perlu memasukkan nilai dalam tanda kutip jika berupa angka.

In [0]:
#@markdown ##Memfilter berdasarkan beberapa nilai
#@markdown Memfilter berdasarkan ‘Indonesia’ membantu Anda menyederhanakan data yang dilihat, sehingga membantu Anda lebih mudah menganalisis data yang diperlukan. Namun demikian, hasil yang Anda dapatkan menggunakan filter ini mungkin masih memuat terlalu banyak informasi yang harus dibandingkan pada saat yang sama. Anda dapat melihat banyak jenis pembangkit listrik dalam kolom 'primary_fuel' yang dicantumkan dalam basis data ini. Terdapat juga berbagai kolom lain yang dapat digunakan untuk memfokuskan analisis Anda pada bagian tertentu data sesuai keinginan. Anda dapat menambahkan lebih banyak filter agar Anda dapat mengambil sebagian kecil data sebagai fokus. Untuk menambahkan beberapa kondisi pada perintah WHERE Anda, gabungkan kondisi dengan AND, seperti contoh di bawah ini:

#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT {columns} FROM
#@markdown {table_name} WHERE {condition1} AND {condition2} AND {condition3}

#@markdown Anda dapat menambahkan sebanyak mungkin kondisi yang diinginkan saat memfilter data.
#@markdown ###Contoh 1:
#@markdown Apabila Anda hanya ingin menampilkan data pembangkit listrik tenaga batu bara di Indonesia, Anda dapat menggunakan SQL query berikut:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' LIMIT 5
#@markdown ###Contoh 2:
#@markdown Sekarang, jika Anda hanya ingin menampilkan pembangkit listrik tenaga batu bara di Indonesia dengan kapasitas di atas 100 MW, Anda dapat memodifikasi SQL query menjadi:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' AND capacity_mw>100 LIMIT 5
#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))
  
r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
  
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Queries yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' AND capacity_mw>100 LIMIT 5
#@markdown <br>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' AND capacity_mw>100 LIMIT 5

#Mengurutkan

In [0]:
#@markdown ##Mengurutkan hasil Anda
#@markdown Setelah Anda mengetahui cara memfilter hanya pembangkit listrik tenaga batu bara di Indonesia, Anda mungkin ingin mengetahui pembangkit listrik terbesar dan terkecil. Untuk melakukannya, Anda dapat mengurutkan SQL query dengan menambahkan perintah ORDER BY.
#@markdown ###Contoh:
#@markdown Anda dapat melihat bahwa terdapat kolom bertuliskan 'capacity_mw' yang menyimpan kapasitas pembangkit listrik tersebut. Anda telah memfilter data sesuai keinginan (pembangkit listrik tenaga batu bara di Indonesia). Sekarang, Anda dapat mengurutkannya dengan kolom 'capacity_mw'. Untuk menampilkan hasil dengan nilai tertinggi pada bagian atas, gunakan perintah DESC, dimana baris semakin ke bawah, nilai akan semakin mengecil. Untuk melakukannya, Anda dapat menggunakan perintah SQL berikut:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' ORDER BY capacity_mw DESC LIMIT 10

#@markdown Sama halnya, jika Anda ingin melihat pembangkit listrik tenaga batu bara di Indonesia dengan nilai terkecil paling atas, Anda dapat mengurutkan kolom 'capacity_mw' dengan perintah ASC, dimana baris semakin ke bawah, semakin besar nilainya. Dalam hal ini, gunakan perintah SQL berikut ini:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' ORDER BY capacity_mw ASC LIMIT 10
#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))

r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Query yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' ORDER BY capacity_mw DESC LIMIT 10 
#@markdown <br>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' ORDER BY capacity_mw ASC LIMIT 10

#Menghitung Statistik

In [0]:
#@markdown Sekarang, Anda mengetahui bagaimana cara menulis query untuk memfilter bagian dataset khusus yang ingin ditampilkan. Anda dapat menjalankan pernyataan SQL yang memfilter pembangkit listrik tenaga batu bara di Indonesia

#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT name, primary_fuel, country_long, commissioning_year, capacity_mw FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal' 

#@markdown Alih-alih menampilkan daftar semua pembangkit listrik tenaga batu bara di Indonesia, Anda mungkin hanya ingin mendapatkan rata-rata kapasitas pembangkit listrik tenaga batu bara di Indonesia. Untuk melakukannya, gunakan fungsi AVG() pada salah satu kolom daripada memilih semua kolom yang ada. Sebagai contoh, Anda ingin melihat kapasitas yang disimpan dalam kolom 'capacity_mw'.
#@markdown
#@markdown Anda dapat menulis pernyataan query yang baru digunakan untuk memfilter lokasi dan waktu yang benar, daripada menggunakan SELECT *, gunakan SELECT AVG(ppm).

#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))
  
r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Queries yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT AVG(capacity_mw) FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal'

In [0]:
#@markdown ##Statistik Lain
#@markdown Rata-rata (AVG) bukan satu-satunya fungsi statistik yang bisa Anda hitung dari tabel. Anda juga dapat menghitung jumlah maksimal (MAX), jumlah minimal (MIN), jumlah keseluruhan (SUM), atau menghitung banyak entri dalam tabel (COUNT).

#@markdown Coba hitung beberapa statistik berikut.

#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))


r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Query yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT MAX(capacity_mw) FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal'
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT MIN(capacity_mw) FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal'
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT COUNT(capacity_mw) FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal'
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT SUM(capacity_mw) FROM Powerwatch WHERE country_long='Indonesia' AND primary_fuel='Coal'

#Latihan

In [0]:
#@markdown ##Uji Pengetahuan SQL Anda
#@markdown Tulis pernyataan SQL queries untuk menjawab pertanyaan di bawah:
#@markdown * Dimanakah pembangkit listrik tenaga air terbesar di Indonesia?
#@markdown * Dimanakah pembangkit listrik tenaga air terbesar di dunia?
#@markdown * Berapa rata-rata kapasitas pembangkit listrik tenaga air di Indonesia?
#@markdown 
#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))


r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)


#Lembar Bantuan
Tutorial ini telah memberikan ikhtisar dasar cara menyusun SQL queries. Berikut adalah tinjauan singkat SQL queries yang telah dipelajari:

**Lihat semua atau beberapa kolom dataset dengan query dasar**: SELECT {columns} FROM {dataset_id}<br>
**Batasi jumlah baris yang Anda lihat dari dataset**: SELECT {columns} FROM {dataset_id} LIMIT {number_of_rows}<br>
**Memfilter data pada beberapa kolom**: SELECT {columns} FROM {table_name} WHERE {column_name} {<, <=, >, >=, =, !=} {value}<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Menerapkan beberapa filter**: SELECT {columns} FROM {dataset_id} WHERE {condition1} AND {condition2} AND {condition3}<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Filter waktu**: SELECT {columns} FROM {dataset_id} WHERE {datetime_column} {<, <=, >, >=, =, !=} {YYYY-MM-DD}T{HH:MI:SS}Z<br>
**Mengurutkan dataset berdasarkan dari nilai terbesar atau terkecil**: SELECT {columns} FROM {dataset_id} ORDER BY {column_name} {ASC, DESC}<br>
**Statistik**: SELECT {function}({column_name}) FROM {dataset_id}<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Fungsi Statistik yang tersedia**: AVG(), SUM(), MIN(), MAX(), COUNT()

Anda dapat menggunakan SQL query untuk mengeluarkan data sesuai keinginan Anda dari API Resource Watch menggunakan Resource Watch Query Tool.

#Tutorial Opsional

In [0]:
#@markdown ##Memfilter berdasarkan waktu
#@markdown Kolom tanggal dapat difilter dengan cara yang sama seperti filter kolom lain, namun perlu diperhatikan untuk memastikan dalam format yang benar. Kolom tanggal memiliki format berikut: '{date}T{time}Z' saat {date} harus diganti dengan tanggal dalam format YYYY-MM-DD, dan {time} harus diganti dengan waktu 24-jam dalam format HH:MI:SS. Format waktu 24 jam sering kali menggunakan sistem Waktu Universal Terkoordinasi (UTC), namun Anda harus selalu memeriksa sumber dataset untuk memastikannya. Misalnya, Anda harus memfilter data untuk hanya menampilkan informasi dari tengah malam 22 Juli 2019 dengan query ini:

#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT {columns} FROM {table_name} WHERE {datetime_column} = '2019-07-22T00:00:00Z'

#@markdown Query apa pun tanpa jam yang ditentukan akan dianggap dimulai dari tengah malam. Jadi, jika Anda mengosongkan bagian jam, contoh query ini akan tetap berfungsi, karena tengah malam (T00:00:00Z) tersirat dalam query ini:

#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT {columns} FROM {table_name} WHERE {datetime_column} = '2019-07-22'
#@markdown 
#@markdown Jadi, jika Anda mencari waktu selain tengah malam pada hari tertentu, pastikan untuk menyertakan bagian jam dalam filter Anda. Apabila tidak, hanya data dari tengah malam yang akan dikeluarkan.
#@markdown 
#@markdown
#@markdown **Catatan**: Tidak semua dataset memuat bidang tanggal. Anda harus memeriksa dataset pada jendela pratinjau untuk melihat jika bidang tanggal disertakan sebelum mengetahui jika Anda dapat memfilter berdasarkan tanggal. 
#@markdown ###Contoh:
#@markdown Apabila dataset memiliki kolom yang mencantumkan waktu, Anda dapat mengembangkan queries untuk menganalisis dataset berdasarkan waktu dan tanggal. Salah satu contoh dataset yang mencantumkan data dengan catatan waktu berbeda adalah Kualitas Udara: Kumpulan Data Pengukuran Stasiun PM2.5, yang memiliki Dataset ID berikut: 
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cit003anrt-Air-Quality-Measurements-PM-25
#@markdown 
#@markdown Data diperbarui beberapa kali sehari. Karena lokasi penghitungan memiliki waktu tertentu yang disertakan dan dilaporkan sebagai bidang tanggal, Anda dapat memfilter berdasarkan jam dan tanggal. Pertama, Anda harus melihat semua kolom untuk mengetahui kolom mana yang memberikan informasi tanggal.
#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))

r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Queries yang disarankan**:
#@markdown 
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT * FROM cit003anrt-Air-Quality-Measurements-PM-25 LIMIT 2

In [0]:

#@markdown ##Memfilter berdasarkan waktu (lanjutan)
#@markdown Anda dapat melihat bahwa kolom 'utc' memiliki informasi tanggal dari dataset ini. Filter semua pengukuran dari tanggal 2019-07-22 pada pukul 18:00:00 UTC dan cetak lima hasil pertama. Untuk contoh ini, Anda juga akan memilih bagian kolom yang lebih bermanfaat.
#@markdown ###Masukkan API query di bawah ini dan jalankan pernyataan SQL untuk melihat hasilnya.
query ="" #@param {type:"string"}
sql_query = urllib.parse.quote(query)
url = 'https://api.resourcewatch.org/query?sql={}'.format(sql_query)
print('Data Query Link: {}'.format(url))
download_url = 'https://api.resourcewatch.org/download?sql={}'.format(sql_query)
print('Data Download Link: {}'.format(download_url))

r = requests.get(url)

if r.status_code == 200:
  df = pd.DataFrame.from_dict(r.json()['data'])
  print("Hasil query:")
  display(df)
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **Queries yang disarankan**:
#@markdown 
#@markdown SELECT city, country, location, ppm, unit, utc FROM cit003anrt-Air-Quality-Measurements-PM-25 WHERE utc = '2019-07-22T18:00:00Z' LIMIT 5

#Tutorial Berikutnya

In [0]:
#@markdown ##Memfilter data kosong

In [0]:
#@markdown ##Memfilter saat nilai data memiliki spasi atau salah ketik



In [0]:
#@markdown ##Mencocokkan tabel dengan shapefile negara WRI


Apabila Anda ingin membahas di luar tutorial ini dan mempelajari SQL queries secara lebih mendalam, lihat dokumentasi sumber di sini: https://www.postgresql.org/.